# trajectory_buoy_thredds

compare wave buoy path data with what parcels says

compare the path of a buoy with a particle simulation on thredds data

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
from parcels import FieldSet
import xarray as xr

import utils
from parcels_utils import buoycsv_to_particleds, HFRGrid
from parcels_sim import simulation, generate_sim_gif
from plot_utils import draw_trajectories, draw_points_fieldset
from thredds_utils import get_thredds_dataset, thredds_data, retrieve_dataset, thredds_data

In [ ]:
# 5-01 west coast (2009 February, too old for thredds)
# 590-09 west coast (2018 October)
# 704-02 west coast
# 689-02 west coast
# 612-08 east coast
# 731-02 east coast
buoy_id = "704-02"
csv_path = utils.WAVEBUOY_DATA_DIR / f"wavebuoy_{buoy_id}.csv"
target_resolution = 6
start_offset = 95
buoy_ds = buoycsv_to_particleds(csv_path)
buoy_ds

In [ ]:
# save csv into netcdf, plot the trajectory for visualization
filename = os.path.basename(csv_path)
nc_path = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}.nc"
buoy_ds.to_netcdf(nc_path)
draw_trajectories([buoy_ds], [buoy_id], padding=0.1)

In [ ]:
# time interval between each recorded position in seconds
buoy_timestamps = buoy_ds["time"].values[0].astype("datetime64[s]")
np.diff(buoy_timestamps)

In [ ]:
buoy_lat_rng = (buoy_ds["lat"].values[0].min(), buoy_ds["lat"].values[0].max())
buoy_lon_rng = (buoy_ds["lon"].values[0].min(), buoy_ds["lon"].values[0].max())
buoy_lat_rng, buoy_lon_rng

In [ ]:
# get a single timestep from the thredds dataset and plot the buoy path on top of the field
# a single timestep is used so there is way less data to retrieve
# this is also to check whether the buoy path is actually in a region with recorded data
single_step = get_thredds_dataset("buoy", target_resolution, (np.datetime64("2020-01-01"), np.datetime64("2020-01-01T01")), buoy_lat_rng, buoy_lon_rng, inclusive=True)
single_step_grid = HFRGrid(single_step)
draw_points_fieldset(buoy_ds["lat"].values[0], buoy_ds["lon"].values[0], 0, single_step_grid, domain=single_step_grid.get_domain(), line=True)

In [ ]:
start = buoy_timestamps[0]
end = buoy_timestamps[-1]
simulation_dataset = get_thredds_dataset("buoy", target_resolution, (start, end), buoy_lat_rng, buoy_lon_rng, inclusive=True, padding=0.05)
simulation_dataset

In [ ]:
# to work with a much smaller time range, so loading data doesn't take forever
grid = HFRGrid(simulation_dataset.sel(time=slice(buoy_timestamps[0], buoy_timestamps[10])), target_resolution)
domain = grid.get_domain()
domain

In [ ]:
domain["W"] = -117.85
domain["E"] = -117.65
domain["S"] = 32.35
domain["N"] = 32.9

In [ ]:
draw_points_fieldset(buoy_ds["lat"].values[0][start_offset:], buoy_ds["lon"].values[0][start_offset:], 0, grid, domain=domain, line=True)

In [ ]:
# find a starting point of the buoy actually in the vector field
start_pos = None
start_time = None
start_time_idx = None
end_time = None
end_time_idx = None
print(buoy_ds.dims["obs"])
for i in range(start_offset, buoy_ds.dims["obs"]):
    time = buoy_ds["time"].values[0, i]
    lat = buoy_ds["lat"].values[0, i]
    lon = buoy_ds["lon"].values[0, i]
    # time is out of bounds, but this is fine since we are just
    # checking for the existence of currents, not value
    u, v = grid.get_closest_current(time, lat, lon)
    if np.isnan(u) or np.isnan(v) or abs(u) + abs(v) == 0:
        if start_pos is not None:
            end_time_idx = i
            end_time = time
            break
    elif start_pos is None:
        start_time_idx = i
        start_pos = [lat, lon]
        start_time = time
    if end_time_idx is None and i == buoy_ds.dims["obs"] - 1:
        end_time_idx = i
        end_time = time
start_time_idx, end_time_idx, start_time, end_time, start_pos

In [ ]:
buoy_ds_selected = buoy_ds.sel(obs=slice(start_time_idx, end_time_idx))
nc_path_selected = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}_selected.nc"
buoy_ds_selected.to_netcdf(nc_path_selected)
draw_points_fieldset(buoy_ds_selected["lat"].values[0], buoy_ds_selected["lon"].values[0], 0, grid, domain=domain, line=True)

In [ ]:
parcels_cfg = {
    "time_range": [start_time, end_time],
    "repeat_dt": -1,
    "particles_per_dt": -1,
    "max_variation": 0.0,
    "spawn_points": [
        start_pos
    ],
    "random_spawn": True,
    "simulation_dt": 300,
    "snapshot_interval": 7200,
    "save_snapshots": True,
    "shown_domain": domain
}

In [ ]:
start_time_floor, end_time_ceil = utils.expand_time_rng((start_time, end_time), "h")
grid = HFRGrid(simulation_dataset.sel(time=slice(start_time_floor, end_time_ceil)))

In [ ]:
grid.xrds.to_netcdf(utils.CURRENT_NETCDF_DIR / f"thredds_buoy_currents_{buoy_id}.nc")

In [ ]:
pfile_path, snap_path = simulation("thredds_buoy", grid, parcels_cfg)

In [ ]:
generate_sim_gif(
    snap_path,
    utils.PICUTRE_DIR / "thredds_buoy.gif",
    25
)

In [ ]:
plot_path = utils.create_path(utils.PICUTRE_DIR / "buoy")
draw_trajectories([nc_path_selected, pfile_path], [buoy_id, "Simulated path"], legend=True, titlestr=f"Comparison from {np.datetime64(start_time, 's')} to {np.datetime64(end_time, 's')}\nResolution {target_resolution}",savefile=plot_path / f"buoy_comp_{buoy_id}.png")